In [23]:
import sqlite3
from sqlite3 import Error
import random
import faker

# Initialize faker generator
fake = faker.Faker()

def create_connection():
    conn = None;
    try:
        conn = sqlite3.connect('Retail.db') # creates a SQLite database named Retail.db
        print(sqlite3.version)
    except Error as e:
        print(e)
    
    if conn:
        return conn

def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def create_database():
    database = r"Retail.db"
 
    sql_create_customers_table = """ CREATE TABLE IF NOT EXISTS Customers (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        email text NOT NULL
                                    ); """

    sql_create_products_table = """ CREATE TABLE IF NOT EXISTS Products (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    price real NOT NULL
                                );"""

    sql_create_orders_table = """ CREATE TABLE IF NOT EXISTS Orders (
                                    id integer PRIMARY KEY,
                                    customer_id integer NOT NULL,
                                    date text NOT NULL,
                                    FOREIGN KEY (customer_id) REFERENCES Customers (id)
                                );"""

    sql_create_orderDetails_table = """ CREATE TABLE IF NOT EXISTS OrderDetails (
                                        order_id integer NOT NULL,
                                        product_id integer NOT NULL,
                                        quantity integer NOT NULL,
                                        FOREIGN KEY (order_id) REFERENCES Orders (id),
                                        FOREIGN KEY (product_id) REFERENCES Products (id)
                                    );"""

    # create a database connection
    conn = create_connection()

    # create tables
    if conn is not None:
        # create customers table
        create_table(conn, sql_create_customers_table)

        # create products table
        create_table(conn, sql_create_products_table)

        # create orders table
        create_table(conn, sql_create_orders_table)

        # create order details table
        create_table(conn, sql_create_orderDetails_table)
    else:
        print("Error! cannot create the database connection.")
        
    return conn


def main():

    conn = create_database()

    # insert random data into the tables
    for _ in range(1000):
        # insert into Customers table
        conn.execute("INSERT INTO Customers(name, email) VALUES(?,?)", (fake.name(), fake.email()))
        
        # insert into Products table
        conn.execute("INSERT INTO Products(name, price) VALUES(?,?)", (fake.catch_phrase(), round(random.uniform(10.5, 200.5), 2)))
        
        # insert into Orders table
        conn.execute("INSERT INTO Orders(customer_id, date) VALUES(?,?)", (random.randint(1,1000), fake.date()))
        
        # insert into OrderDetails table
        conn.execute("INSERT INTO OrderDetails(order_id, product_id, quantity) VALUES(?,?,?)", (random.randint(1,1000), random.randint(1,1000), random.randint(1,10)))

    conn.commit()

if __name__ == '__main__':
    main()



2.6.0


In [24]:
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [33]:
%sql sqlite:///Retail.db

In [34]:
%sql SELECT * FROM Orders limit 10

   sqlite:///Retail
 * sqlite:///Retail.db
Done.


id,customer_id,date
1,278,1988-02-14
2,552,1974-11-24
3,107,1976-05-25
4,849,2015-03-28
5,334,2009-01-17
6,427,2007-04-21
7,119,2016-07-27
8,52,1997-04-19
9,508,2022-01-02
10,907,1996-01-02


In [47]:
%sql SELECT CustomerID, SUM(Quantity * Price) AS TotalSpent FROM Sales GROUP BY CustomerID ORDER BY TotalSpent DESC LIMIT 1;


   sqlite:///Retail
 * sqlite:///Retail.db
(sqlite3.OperationalError) no such table: Sales
[SQL: SELECT CustomerID, SUM(Quantity * Price) AS TotalSpent FROM Sales GROUP BY CustomerID ORDER BY TotalSpent DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
